# Pregunta 3

### Chequeos previos de performance

Para evitar tiempos de entrenamiento muy extensos es recomendable utilizar la **GPU** disponible en el sistema en lugar de la **CPU**, ya que posee la habilidad de realizar cálculos matemáticos de manera más eficiente. A continuación se verifica que se esté efectivamente utilizando la GPU en lugar de la CPU.

In [1]:
from tensorflow.python.client import device_lib
print(device_lib.list_local_devices())

C:\Users\Felipe\Anaconda3\lib\site-packages\h5py\__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


[name: "/device:CPU:0"
device_type: "CPU"
memory_limit: 268435456
locality {
}
incarnation: 6714478722248921555
, name: "/device:GPU:0"
device_type: "GPU"
memory_limit: 3178453401
locality {
  bus_id: 1
  links {
  }
}
incarnation: 4900222509545167249
physical_device_desc: "device: 0, name: GeForce GTX 980, pci bus id: 0000:01:00.0, compute capability: 5.2"
]


# Generación del dataset

In [2]:
from keras.datasets import cifar10
import keras
label_names = ['airplane', 'automobile', 'bird', 'cat', 'deer', 'dog', 'frog', 'horse', 'ship', 'truck']
(x_train, y_train), (x_original_test, y_original_test) = cifar10.load_data()

y_train = keras.utils.to_categorical(y_train)
y_original_test = keras.utils.to_categorical(y_original_test, num_classes=10)

Using TensorFlow backend.


# Generación de conjuntos de validación, test y entrenamiento

In [3]:
from sklearn.model_selection import train_test_split

x_test, x_val, y_test, y_val = train_test_split(x_original_test, y_original_test, test_size=0.5, random_state=42)


print("Train samples: %d" % len(x_train))
print("Test samples: %d" % len(x_test))
print("Validation samples: %d" % len(x_val))

Train samples: 50000
Test samples: 5000
Validation samples: 5000


# Cambios de dimensionalidad

In [4]:
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation, Flatten
from keras.layers import Conv2D, MaxPooling2D

def define_convolutional(filter_size=3):
    model = Sequential()
    model.add(Conv2D(64, (filter_size, filter_size), padding='same', input_shape=x_train.shape[1:]))
    model.add(Activation('relu'))
    model.add(MaxPooling2D(pool_size=(2, 2)))
    model.add(Conv2D(64, (filter_size, filter_size), padding='same'))
    model.add(Activation('relu'))
    model.add(MaxPooling2D(pool_size=(2, 2)))
    model.add(Flatten())
    model.add(Dense(512))
    model.add(Activation('relu'))
    model.add(Dense(10))
    model.add(Activation('softmax'))
    return model
model = define_convolutional()
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_1 (Conv2D)            (None, 32, 32, 64)        1792      
_________________________________________________________________
activation_1 (Activation)    (None, 32, 32, 64)        0         
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 16, 16, 64)        0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 16, 16, 64)        36928     
_________________________________________________________________
activation_2 (Activation)    (None, 16, 16, 64)        0         
_________________________________________________________________
max_pooling2d_2 (MaxPooling2 (None, 8, 8, 64)          0         
_________________________________________________________________
flatten_1 (Flatten)          (None, 4096)              0         
__________

In [ ]:
# Pending to visualize the network

d) Entrene la CNN definida en c) utilizando SGD. En este dataset, una tasa de aprendizaje “segura” es  η=104η=104  o inferior, pero durante las primeras epochs el entrenamiento resulta demasiado lento. Para resolver el problema aprenderemos a controlar la tasa de aprendizaje utilizada en el entrenamiento. Implemente la siguiente idea: deseamos partir con una tasa de aprendizaje  η=103η=103  y dividir por 2 ese valor cada 10 epochs. Suponga además que no queremos usar una tasa de aprendizaje menor a  η=105η=105 . Construya un gráfico que muestre los errores de entrenamiento, validación y pruebas como función del número de “epochs”, entrene con 25 epochs.

# Entrenamiento de red usando SGD

Realizamos el entrenamiento con un decay customizado definido en `step_decay`

In [ ]:
from keras.optimizers import SGD, rmsprop
from keras.callbacks import LearningRateScheduler
import math
def step_decay(epoch):
    initial_lrate = 0.001
    lrate = initial_lrate * math.pow(0.5, math.floor((1+epoch)/5))
    lrate = max(lrate,0.00001)
    return lrate

# Entrenamiento de la red
model = define_convolutional()
sgd = SGD(lr=0.0, momentum=0.9, decay=0.0)
lrate = LearningRateScheduler(step_decay)
model.compile(optimizer=sgd, loss="categorical_crossentropy", metrics=["acc"])
history = model.fit(x_train, y_train, batch_size=32, epochs=25, validation_data=(x_test,y_test), shuffle=True, callbacks=[lrate])

### Resultados

# Entrenamiento de red utilizando RMSProp

In [ ]:
from keras.optimizers import SGD, rmsprop
model = define_convolutional()
opt = rmsprop(lr=0.001, decay=1e-6)
model.compile(optimizer=opt, loss="categorical_crossentropy", metrics=["acc"])
history = model.fit(x_train, y_train,batch_size=32,epochs=25, validation_data=(x_test, y_test),shuffle=True)

f) Evalúe el efecto de modificar el tamaño de los filtros (de convolución) reportando la sensibilidad del error de pruebas a estos cambios en dos tipos de arquitecturas, una profunda y otra no. Presente un gráfico o tabla resumen. Por simplicidad entre durante sólo 15-20 epochs.

# Efecto del tamaño de los filtros de convolución

In [6]:
def define_shallow_convolutional(filter_size=3):
    model = Sequential()
    model.add(Conv2D(64, (filter_size, filter_size), padding='same', input_shape=x_train.shape[1:]))
    model.add(Activation('relu'))
    model.add(MaxPooling2D(pool_size=(2, 2)))
    model.add(Flatten())
    model.add(Dense(512))
    model.add(Activation('relu'))
    model.add(Dense(10))
    model.add(Activation('softmax'))
    return model

In [ ]:
import numpy as np

filter_results = {
    "shallow": [],
    "deep": []
}

filter_sizes = np.linspace(2, 9, dtype = int)

for filter_size in filter_sizes:
    print("Training with filter size %d" % filter_size)
    # Use the shallow first
    model = define_shallow_convolutional(filter_size=filter_size)
    model.compile(optimizer=sgd, loss="categorical_crossentropy", metrics=["acc"])
    history = model.fit(x_train, y_train,batch_size=32,epochs=20, validation_data=(x_test, y_test),shuffle=True, verbose=False)
    filter_results["shallow"].append(history)
    
    # Deep one
    model = define_convolutional(filter_size=filter_size)
    model.compile(optimizer=sgd, loss="categorical_crossentropy", metrics=["acc"])
    history = model.fit(x_train, y_train,batch_size=32,epochs=20, validation_data=(x_test, y_test),shuffle=True, verbose=False)
    filter_results["deep"].append(history)

g)Se ha sugerido que la práctica bastante habitual de continuar una capa convolucional con una capa de pooling puede generar una reducción prematura de las dimensiones del patrón de entrada. Experimente con una arquitectura del tipo  C×C×P×C×C×P×F×F. Use 64 filtros para las primeras 2 capas convolucionales y 128 para las últimas dos. Reflexione sobre qué le parece más sensato: ¿mantener el tamaño de los filtros usados anteriormente? o ¿usar filtros más grandes en la segunda capa convolucional y más pequeños en la primera? o ¿usar filtros más pequeños en la segunda capa convolucional y más grandes en la primera? Hint: con esta nueva arquitectura debiese superar el 70% de accuracy (de validación/test) antes de 5 epochs, pero la arquitectura es más sensible a overfitting por lo que podrı́a ser conveniente agregar un regularizador. Como resultado final de esta actividad gráficque los errores de entrenamiento, validación y pruebas como función del número de “epochs” (fijando el máximo en un número razonable como T = 25).



# Doble capa de convolución

In [16]:
def define_convolutional_for_g():
    model = Sequential()
    
    # C
    model.add(Conv2D(64, (3, 3), padding='same', input_shape=x_train.shape[1:]))
    model.add(Activation('relu'))
    
    # C
    model.add(Conv2D(64, (3, 3), padding='same'))
    model.add(Activation('relu'))
    
    # P
    model.add(MaxPooling2D(pool_size=(2, 2)))
    model.add(Dropout(0.25))
    
     # C
    model.add(Conv2D(128, (3, 3), padding='same', input_shape=x_train.shape[1:]))
    model.add(Activation('relu'))
    
    # C
    model.add(Conv2D(128, (3, 3), padding='same'))
    model.add(Activation('relu'))
    
    # P
    model.add(MaxPooling2D(pool_size=(2, 2)))
    model.add(Dropout(0.25))
    
    # F
    model.add(Dense(512))
    model.add(Activation('relu'))
    
    # F
    model.add(Dense(10))
    model.add(Activation('softmax'))
    return model

In [11]:
model = define_convolutional_for_g()
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_13 (Conv2D)           (None, 32, 32, 64)        1792      
_________________________________________________________________
activation_21 (Activation)   (None, 32, 32, 64)        0         
_________________________________________________________________
conv2d_14 (Conv2D)           (None, 32, 32, 64)        36928     
_________________________________________________________________
activation_22 (Activation)   (None, 32, 32, 64)        0         
_________________________________________________________________
max_pooling2d_9 (MaxPooling2 (None, 16, 16, 64)        0         
_________________________________________________________________
dropout_5 (Dropout)          (None, 16, 16, 64)        0         
_________________________________________________________________
conv2d_15 (Conv2D)           (None, 16, 16, 128)       73856     
__________

In [17]:
model = define_convolutional_for_g()
opt = rmsprop(lr=0.001, decay=1e-6)
model.compile(optimizer=opt, loss="categorical_crossentropy", metrics=["acc"])
history = model.fit(x_train, y_train,batch_size=32,epochs=25, validation_data=(x_test, y_test),shuffle=True)

ValueError: Error when checking target: expected activation_50 to have 4 dimensions, but got array with shape (50000, 10)

h)Algunos investigadores, han propuesto que las capas de pooling se pueden reemplazar por capas convoluciones con stride 2. ¿Se reduce dimensionalidad de este modo? Compruébelo verificando los cambios de forma (dimensionalidad) que experimenta un patrón de entrada a medida que se ejecuta un forward-pass. Entrene la red resultante con el método que prefiera, gráficando los errores de entrenamiento, validación y pruebas como función del número de “epochs” (fijando el máximo en un número razonable como T = 25).

# Utilización de convolución con strides

In [ ]:
def define_convolutional_for_h:
    # C
    model.add(Conv2D(128, (3, 3), padding='same'))
    model.add(Activation('relu'))
    
    # C
    model.add(Conv2D(128, (3, 3), padding='same'))
    model.add(Activation('relu'))
    
    # CS
    model.add(Conv2D(64, (3, 3), strides=(2, 2), padding='same'))

i) Una forma interesante de regularizar modelos entrenados para visión artificial consiste en “aumentar” el número de ejemplos de entrenamiento usando transformaciones sencillas como: rotaciones, corrimientos y reflexiones, tanto horizontales como verticales. Explique porqué este procedimiento podrı́a ayudar a mejorar el modelo y el porqué las etiquetas no cambian al aplicar estas operaciones. Evalúe experimentalmente la conveniencia de incorporarlo.

In [ ]:
from keras.preprocessing.image import ImageDataGenerator
datagen = ImageDataGenerator(
    featurewise_center=False, # set input mean to 0 over the dataset
    samplewise_center=False, # set each sample mean to 0
    featurewise_std_normalization=False, # divide inputs by std of the dataset
    samplewise_std_normalization=False, # divide each input by its std
    zca_whitening=False, # apply ZCA whitening
    rotation_range=0, # randomly rotate images (degrees, 0 to 180)
    width_shift_range=0.1, # randomly shift images horizontally (fraction of width)
    height_shift_range=0.1, # randomly shift images vertically (fraction of height)
    horizontal_flip=True, # randomly flip images
    vertical_flip=False) # randomly flip images
datagen.fit(x_train)
model.fit_generator(datagen.flow(x_train, y_train,batch_size=batch_size),steps_per_epoch=x_train.shape[0]// batch_size, epochs=epochs,validation_data=(x_test, y_test))

j)Elija una de las redes entrenadas en esta sección (preferentemente una con buen desempeño) y determine los pares de objetos (por ejemplo “camiones” con “autos”) que la red tiende a confundir. Conjeture el motivo de tal confusión. **Hay que hacer una matriz de confusión**

k)Elija una de las redes entrenadas (preferentemente una con buen desempeño) y visualice los pesos correspondientes a los filtros de la primera capa convolucional. Visualice además el efecto del filtro sobre algunas imágenes de entrenamiento. Repita el proceso para los pesos de la última capa convolucional. Comente.